In [8]:
# pip install weaviate-client cohere langchain

In [11]:
# pip install pdfminer.six unstructured[unstructured_inference] PyMuPDF

In [13]:
# pip install 'PyPDF2<3.0'

In [4]:
# pip install unstructured pdf2image pikepdf unstructured-inference camelot-py[base]

In [5]:
# !apt-get install ghostscript

In [9]:
# !pip install pillow-heif

In [11]:
# !pip install pypdf

In [ ]:
# mkdir data

In [12]:
import csv
import requests
import pandas as pd
import weaviate
import cohere
import pdfplumber
import camelot as cam
from langchain.vectorstores import Weaviate
from langchain_community.embeddings import CohereEmbeddings
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain_community.llms import Cohere
from langchain.document_loaders import TextLoader

In [13]:
WEAVIATE_API_KEY = "WfdvclBO4vWoiOfXCV1DjwOZUNZ08nr3NACb"
WEAVIATE_CLUSTER = "https://weaviate-hackathon-cluster-7l2iio8z.weaviate.network"
COHERE_API_KEY = "QnW41n57tHZl6r5EscOHPg1KbF6YrUAhpE7oJGSA"

**Loading datasets for preprocessing**

In [19]:
# Note: Here, we are trying to analyse financial data. So to get better results, extracting tables from the pdf separately using Camelot package and merging balance sheet, statement of P and L, and cash flow as one CSV file.

In [ ]:
# Extracting tables from pdf

In [ ]:
pdf_hl = cam.read_pdf("ABB India Limited16.pdf",pages="6", flavor="stream")
pdf_hl[0].df.to_csv("Five-year-highlights.csv",index=False)

In [ ]:
pdf_bs = cam.read_pdf("ABB India Limited16.pdf",pages="100", flavor="stream")
pdf_bs[0].df.to_csv("balancesheet.csv",index=False)

In [ ]:
pdf_spl = cam.read_pdf("ABB India Limited16.pdf",pages="101", flavor="stream")
pdf_spl[0].df.to_csv("Statement of Profit and Loss.csv",index=False)

In [ ]:
pdf_cf = cam.read_pdf("ABB India Limited16.pdf",pages="102", flavor="stream")
pdf_cf[0].df.to_csv("cash_flow_statement.csv",index=False)

**# Connecting to weaviate cluster**

In [ ]:
auth_config = weaviate.auth.AuthApiKey(api_key = WEAVIATE_API_KEY)
weaviate_url = WEAVIATE_CLUSTER

In [ ]:
client = weaviate.Client(url = weaviate_url,
additional_headers = {"X-Cohere-Api-Key":COHERE_API_KEY},
auth_client_secret = auth_config)

In [ ]:
client.is_ready()

True

In [ ]:
client.schema.delete_all()

In [ ]:
client.schema.get()

{'classes': []}

In [ ]:
# Note: we are using txt and csv files for creating the knowledge base for LLM (all these tables are extracted from ABB India Limited16.pdf)

1. creating class for Five-year-highlights data

In [ ]:
class_nameIfiveyhighl = "Fiveyearhighlights"

class_obj = {
    "class": class_nameIfiveyhighl,
    "vectorizer": "text2vec-cohere",
    }

client.schema.create_class(class_obj)

In [ ]:
# client.schema.get()

In [ ]:
weaviate_url = WEAVIATE_CLUSTER
csv_file_path = "Five-year-highlights.csv"

In [ ]:
csv_file_path = "Five-year-highlights.csv"
df = pd.read_csv(csv_file_path,index_col="Description")

In [ ]:
# col_names = df.columns

In [ ]:
# df[col_names] = df[col_names].apply(pd.to_numeric)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, Shares capital to Number of permanent employees
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   2016    19 non-null     int64
 1   2015    19 non-null     int64
 2   2014    19 non-null     int64
 3   2013    19 non-null     int64
 4   2012    19 non-null     int64
dtypes: int64(5)
memory usage: 912.0+ bytes


In [ ]:
hl_result_dict = []

for index, row in df.iterrows():
    for year, value in row.items():
        hl_result_dict.append({
            "description": index,
            "year": int(year),
            "value": int(value)
        })

In [ ]:
hl_result_dict[0]

{'description': 'Shares capital', 'year': 2016, 'value': 42}

In [ ]:
# Reference:
# https://weaviate.io/developers/weaviate/manage-data/import
# https://weaviate-python-client.readthedocs.io/en/stable/weaviate.batch.html

In [ ]:
with client.batch as batch:  # Context manager manages batch flushing
    batch.batch_size=10
    batch.dynamic=True
    for data_obj in hl_result_dict:
        batch.add_data_object(
            data_obj,
            class_nameIfiveyhighl
        )

In [ ]:
# print(client.query.aggregate(class_name).with_meta_count().do())

2. creating class for Cash flow statement, Statement of Profit and Loss, Five-year-highlights

In [ ]:
class_bs_cf_spl_tables = "bs_cf_spl_tables"

class_obj = {
    "class": class_bs_cf_spl_tables,
    "vectorizer": "text2vec-cohere",
    }

client.schema.create_class(class_obj)

In [ ]:
csv_cashflow = "cashflowstatement_balancesheet_statementofPnL - Copy.csv"
df_cashflow = pd.read_csv(csv_cashflow,index_col="label")

In [ ]:
df_cashflow.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, Profit before tax to Diluted
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2016    86 non-null     float64
 1   2015    86 non-null     float64
dtypes: float64(2)
memory usage: 2.0+ KB


In [ ]:
cash_flow_dict = []

for index, row in df_cashflow.iterrows():
    for year, value in row.items():
        cash_flow_dict.append({
            "bs_cf_spl_description": index,
            "bs_cf_spl_year": int(year),
            "bs_cf_spl_value": int(value)
        })

In [ ]:
cash_flow_dict[0]

{'bs_cf_spl_description': 'Profit before tax',
 'bs_cf_spl_year': 2016,
 'bs_cf_spl_value': 576}

In [ ]:
with client.batch as batch:  # Context manager manages batch flushing
    batch.batch_size=10
    batch.dynamic=True
    for data_obj in cash_flow_dict:
        batch.add_data_object(
            data_obj,
            class_bs_cf_spl_tables
        )

3. creating class for ABB text data

In [ ]:
class_name_abb_data = "Abbdata"

In [ ]:
class_obj = {
    "class": class_name_abb_data,
    "vectorizer": "text2vec-cohere",
    }

client.schema.create_class(class_obj)

In [ ]:
# client.schema.get()

In [16]:
# # pdf
loader = UnstructuredPDFLoader("ABB India Limited16.pdf")
abb_data = loader.load()

# # txt file
# loader = TextLoader("ABB India Limited16pdf.txt")
# abb_data = loader.load()

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=0)
abb_docs = text_splitter.split_documents(abb_data)

In [ ]:
len(abb_docs)

489

In [ ]:
abb_data_list = []
for chunk in abb_docs:
  data_new  = {}
  data_new["doc_text"] = chunk.page_content
  abb_data_list.append(data_new)

In [ ]:
with client.batch as batch:  # Context manager manages batch flushing
    batch.batch_size=10
    batch.dynamic=True
    for data_obj in abb_data_list:
        batch.add_data_object(
            data_obj,
            class_name_abb_data
        )

In [ ]:
# client.schema.get()

In [ ]:
# abb_data_list

Verifying vector database

In [ ]:
# user_prompt = "Number of permanent employees in the year 2012"
user_prompt = "what was the Profit after tax in the year 2016?"

In [ ]:
results1 = client.query.get("Fiveyearhighlights", ["description", "year", "value"]
).with_near_text(
    {"concepts": [user_prompt]}
).with_limit(5).do()
print(results1)

{'data': {'Get': {'Fiveyearhighlights': [{'description': 'Profit after tax', 'value': 229, 'year': 2014}, {'description': 'Profit after tax', 'value': 137, 'year': 2012}, {'description': 'Profit after tax', 'value': 300, 'year': 2015}, {'description': 'Profit after tax', 'value': 179, 'year': 2013}, {'description': 'Profit after tax', 'value': 376, 'year': 2016}]}}}


In [ ]:
results2 = client.query.get("Abbdata", ["doc_text"]
).with_near_text(
    {"concepts": [user_prompt]}
).with_limit(3).do()
print(results2)

{'data': {'Get': {'Abbdata': [{'doc_text': 'due to operational excellence initiatives, supply chain \n\nefficiencies, focus on project management. Net profit \n\nafter tax was up by 25% at ` 376 crore for the current \n\nyear as compared to ` 300 crore in the previous year. \n\nConsequently the earnings per share for the year 2016 \n\nstood at ` 17.76 per share as compared to ` 14.15 per \n\nshare in the year 2015.\n\nParticulars For the year ended\n\nDecember 31, 2016 December 31, 2015\n\nProfit Before Taxation  576.05 474.59\n\nLess: Provision for Tax  \n\n- Current Tax  229.20 203.71\n\n- Deferred Tax  (29.40) (29.00)\n\nProfit after Tax  376.25 299.88\n\nBalance Brought Forward from last year  109.41 103.90\n\nAmount available for Appropriation  485.66 403.78\n\nAppropriations:  \n\nGeneral Reserve  210.00   150.00\n\nDebenture Redemption Reserve  50.00   50.00'}, {'doc_text': 'year was at ` 11,821 crore, providing visibility to the future \n\nrevenue streams. \n\n\n\n48 ABB INDIA 

In [ ]:
results3 = client.query.get("bs_cf_spl_tables", ["bs_cf_spl_description", "bs_cf_spl_year", "bs_cf_spl_value"]
).with_near_text(
    {"concepts": [user_prompt]}
).with_limit(10).do()
print(results3)

{'data': {'Get': {'Bs_cf_spl_tables': [{'bs_cf_spl_description': 'Profit before tax', 'bs_cf_spl_value': 474, 'bs_cf_spl_year': 2015}, {'bs_cf_spl_description': 'Profit before tax', 'bs_cf_spl_value': 576, 'bs_cf_spl_year': 2016}, {'bs_cf_spl_description': 'Profit for the year', 'bs_cf_spl_value': 299, 'bs_cf_spl_year': 2015}, {'bs_cf_spl_description': 'Profit for the year', 'bs_cf_spl_value': 376, 'bs_cf_spl_year': 2016}, {'bs_cf_spl_description': 'Operating profit before working capital changes', 'bs_cf_spl_value': 797, 'bs_cf_spl_year': 2016}, {'bs_cf_spl_description': 'Revenue from operations (net)', 'bs_cf_spl_value': 8648, 'bs_cf_spl_year': 2016}, {'bs_cf_spl_description': 'Revenue from operations (net)', 'bs_cf_spl_value': 8140, 'bs_cf_spl_year': 2015}, {'bs_cf_spl_description': 'Operating profit before working capital changes', 'bs_cf_spl_value': 795, 'bs_cf_spl_year': 2015}, {'bs_cf_spl_description': 'Net cash flow from operating activities', 'bs_cf_spl_value': 862, 'bs_cf_spl

In [ ]:
# user_prompt = "Gross fixed assets?"

In [ ]:
# results_final = client.query.get("bs_cf_spl_tables", ["bs_cf_spl_description", "bs_cf_spl_year", "bs_cf_spl_value"]
# ).with_near_text(
#     {"concepts": [user_prompt],
#      "classes": [
#          {"name1": "Fiveyearhighlights", "features": ["description", "year", "value"]},
#          {"name2": "Abbdata", "features": ["doc_text"]}
#      ]}
# ).with_limit(10).do()
# print(results_final)

In [ ]:
# response

In [ ]:
co = cohere.Client(COHERE_API_KEY)

In [ ]:
cohere.api_key = COHERE_API_KEY

class_name = "Fiveyearhighlights"
prompt = f"use the data {results1}{results2}{results3} and answer the following question in simple sentence " +user_prompt


In [ ]:
response = co.chat(prompt)


In [ ]:
response.text

'The profit after tax was 376 crore for the year 2016.'

In [ ]:
#_------------------------------------------------------EOF

In [ ]:
def promptresponse(user_prompt):
    results1 = client.query.get("Fiveyearhighlights", ["description", "year", "value"]
                                ).with_near_text(
                                    {"concepts": [user_prompt]}
                                ).with_limit(5).do()

    results2 = client.query.get("Abbdata", ["doc_text"]
                                ).with_near_text(
                                    {"concepts": [user_prompt]}
                                ).with_limit(3).do()

    results3 = client.query.get("bs_cf_spl_tables", ["bs_cf_spl_description", "bs_cf_spl_year", "bs_cf_spl_value"]
                                ).with_near_text(
                                    {"concepts": [user_prompt]}
                                ).with_limit(10).do()

    prompt = f"use the data {results1}{results2}{results3} and answer the following question in simple sentence " +user_prompt

    response = co.chat(prompt)

    return response.text


In [ ]:
user_prompt_0 = "what are the financial red flags for this company?"

In [ ]:
promptresponse(user_prompt_0)

"The company has had generally increasing profitability from 2012 to 2016, with the exception of a drop in 2015. However, it has also experienced increasing finance costs from 2013 to 2016 and net cash flow usage for investing activities increased in 2016 compared to 2015. Net cash flow from operating activities was higher in 2016 than 2015 and net cash flow from financing activities was lower in 2016 than 2015. \n\nThese financial data points do not indicate any significant red flags, but it is always important to consider additional context and factors that may be relevant to forming a fuller picture of the company's financial health."

In [ ]:
user_prompt_1 = "what was the Profit after tax in the year 2016"

In [ ]:
promptresponse(user_prompt_1)

'The profit after tax was 376 crore for the year 2016.'

In [ ]:
user_prompt_2 = "total outstanding dues to micro enterprises and small enterprises 2016"

In [ ]:
promptresponse(user_prompt_2)

'The total outstanding dues to micro and small enterprises in 2016 was 117.68.'

In [ ]:
user_prompt_3 = "summarize Auditor’s Responsibility"

In [ ]:
promptresponse(user_prompt_3)

"The auditor's responsibility is to express an opinion on the company's internal financial controls over financial reporting based on the audit. The audit was conducted in accordance with the Guidance Note on Audit of Internal Financial Controls Over Financial Reporting and the Standards on Auditing to obtain reasonable assurance about whether adequate internal financial controls over financial reporting were provided."

In [ ]:
user_prompt_4 = "information on comapny revenue growth"

In [ ]:
promptresponse(user_prompt_4)

"Here are the revenue growth figures for the company ABB India Limited from 2012 to 2016. The information is taken from the annual reports of the company:\n\n2012: value : 7565\n2013: value : 7722\n2014: value : 7733\n2015: value : 8140\n2016: value : 8648. \n\nIt appears that there was a consistent increase in revenue from operations net and gross over the five-year period with the exception of a slight decrease in 2015 for the operation's gross revenue. \n\nHowever, overall, it seems like the company's revenue from operations net and gross grew significantly from 2012 to 2016. \n\nIt would be advisable to examine the company's financial statements and annual reports for more current information and to make decisions based on the nascent trends foreseen."

In [ ]:
# Creating chat session

In [ ]:
# Create a conversation ID
import uuid
conversation_id = str(uuid.uuid4())

print('Starting the chat. Type "quit" to end.\n')

while True:

    # User message
    message = input("User: ")

    # Typing "quit" ends the conversation
    if message.lower() == 'quit':
        print("Ending chat.")
        break

    results1 = client.query.get("Fiveyearhighlights", ["description", "year", "value"]).with_near_text({"concepts": [message]}).with_limit(5).do()

    results2 = client.query.get("Abbdata", ["doc_text"]).with_near_text({"concepts": [message]}).with_limit(3).do()


    preamble_override = f"Financial Analyst chatbot.Use the data {results1}{results2} and answer the following question in simple sentence " +message
    # Chatbot response
    response = co.chat(message=message,
                        preamble_override=preamble_override,
                        stream=True,
                        conversation_id=conversation_id,
                        return_chat_history=True)

    print("Chatbot: ", end='')

    for event in response:
        if event.event_type == "text-generation":
            print(event.text, end='')
    print("\n")

Starting the chat. Type "quit" to end.

User: give the total net cash flow from operating activities in the year 2015 to 2016
Chatbot: The total net cash flow from operating activities for the years 2015 and 2016 is 1,189.16 Crores and 573.59 Crores respectively. 
Did you want to know more about the cash flow statement or ABB India Limited? 

I can provide more information or specific data if needed.

User: Unrealised exchange loss / (gains) (net)
Chatbot: According to the data within my parameters, the net amount of the unrealised exchange loss on the cash and cash equivalents for the years mentioned cannot be provided.

Is there anything else I can help you with?  I can provide more information or specific data if needed.

User: quit
Ending chat.
